In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder\
    .appName('book1')\
    .getOrCreate()

21/10/27 17:59:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

retail_schema = StructType([
    StructField('InvoiceNo', LongType()),
    StructField('StockCode', StringType()),
    StructField('Description', StringType()),
    StructField('Quantity', IntegerType()),
    StructField('InvoiceDate', TimestampType()),
    StructField('UnitPrice', FloatType()),
    StructField('CustomerID', FloatType()),
    StructField('Country', StringType()),
])

df = spark.read\
    .schema(retail_schema)\
    .format('csv')\
    .option('header', True)\
    .load('/Users/mrot/workspace/trade-bigdata/Spark-The-Definitive-Guide/data/retail-data/all/online-retail-dataset.csv')


In [6]:
df.rdd.getNumPartitions()

11

In [15]:

def f(index, it):
    import itertools
    return map(lambda el: f'{index}_{el["Country"]}', itertools.islice(it, 0, 15))

df.repartition(3, 'Country').rdd.mapPartitionsWithIndex(f).collect()

['0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Australia',
 '0_Norway',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '1_United Kingdom',
 '2_France',
 '2_France',
 '2_France',
 '2_France',
 '2_France',
 '2_France',
 '2_France',
 '2_France',
 '2_France',
 '2_France',
 '2_France',
 '2_France',
 '2_France',
 '2_France',
 '2_France']

In [19]:

def f(index, it):
    from collections import Counter
    from operator import attrgetter
    return index, dict(Counter(map(attrgetter('Country'), it)))

df.repartition(3, 'Country').rdd.mapPartitionsWithIndex(f).collect()

[0,
 {'Australia': 1259,
  'Norway': 1086,
  'EIRE': 8196,
  'Switzerland': 2002,
  'Poland': 341,
  'Portugal': 1519,
  'Japan': 358,
  'Iceland': 182,
  'Channel Islands': 758,
  'Israel': 297,
  'Greece': 146,
  'Hong Kong': 288,
  'Singapore': 229,
  'USA': 291,
  'European Community': 61},
 1,
 {'United Kingdom': 495478,
  'Netherlands': 2371,
  'Germany': 9495,
  'Italy': 803,
  'Lithuania': 35,
  'Cyprus': 622,
  'Austria': 401,
  'Saudi Arabia': 10,
  'Czech Republic': 30,
  'Malta': 127},
 2,
 {'France': 8557,
  'Spain': 2533,
  'Belgium': 2069,
  'Denmark': 389,
  'Sweden': 462,
  'Finland': 695,
  'Bahrain': 19,
  'Lebanon': 45,
  'United Arab Emirates': 68,
  'Canada': 151,
  'Unspecified': 446,
  'Brazil': 32,
  'RSA': 58}]

21/10/27 18:36:36 ERROR RetryingBlockFetcher: Exception while beginning fetch of 1 outstanding blocks (after 1 retries)
java.io.IOException: Failed to connect to /10.92.248.4:65088
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:287)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:218)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:123)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.fetchAllOutstanding(RetryingBlockFetcher.java:153)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.lambda$initiateRetry$0(RetryingBlockFetcher.java:181)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1130)
	at java.base/ja

In [21]:
df.limit(20).repartition(3).rdd.glom().collect()

[[Row(InvoiceNo=536367, StockCode='21777', Description='RECIPE BOX WITH METAL HEART', Quantity=4, InvoiceDate=None, UnitPrice=7.949999809265137, CustomerID=13047.0, Country='United Kingdom'),
  Row(InvoiceNo=536367, StockCode='22622', Description='BOX OF VINTAGE ALPHABET BLOCKS', Quantity=2, InvoiceDate=None, UnitPrice=9.949999809265137, CustomerID=13047.0, Country='United Kingdom'),
  Row(InvoiceNo=536367, StockCode='84879', Description='ASSORTED COLOUR BIRD ORNAMENT', Quantity=32, InvoiceDate=None, UnitPrice=1.690000057220459, CustomerID=13047.0, Country='United Kingdom'),
  Row(InvoiceNo=536366, StockCode='22633', Description='HAND WARMER UNION JACK', Quantity=6, InvoiceDate=None, UnitPrice=1.850000023841858, CustomerID=17850.0, Country='United Kingdom'),
  Row(InvoiceNo=536367, StockCode='22745', Description="POPPY'S PLAYHOUSE BEDROOM ", Quantity=6, InvoiceDate=None, UnitPrice=2.0999999046325684, CustomerID=13047.0, Country='United Kingdom'),
  Row(InvoiceNo=536367, StockCode='8496

In [24]:
df.withColumn('isBig', df['Quantity']>3).\
repartition(3).write.format('csv').option('header', 'True').partitionBy('Country', 'isBig').save('/Users/mrot/Downloads/out')